In [42]:
import sys
import os
from pathlib import Path
#
path_root = Path(os.path.abspath(''))
sys.path.insert(1, os.path.join(path_root))
#
path_root2 = Path(os.path.abspath('')).parent
sys.path.insert(1, os.path.join(path_root2))
import talib
import sqlite3 as sqlite3
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import matplotlib.font_manager as fm
from mplfinance.original_flavor import candlestick_ohlc
from util.util import get_ucodes

In [43]:
path_sqlite = os.path.join(path_root, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
if not os.path.exists(path_sqlite):
    path_sqlite = os.path.join(path_root2, 'data', 'aastock', 'sqlite', 'hk-marketwatch.db')
print(path_sqlite)
conn = sqlite3.connect(path_sqlite)
cursor = conn.cursor()
conn.row_factory = lambda cursor, row: row[0]
no_days = 60

D:\PycharmProjects\scrapy-001\data\aastock\sqlite\hk-marketwatch.db


In [44]:
data1 = {}
ucodes = ['hsi', 'hsce', 'hstech', 'ssec', 'csi300', 'sp500', 'dji', 'ixic']
for ucode in ucodes:
    sql = """SELECT t.code, t.lot, t.nmll, t.stime, t.high, t.low, t.open, t.close, t.volume
                FROM (SELECT n.code, n.lot, n.nmll, c.stime, c.high, c.low, c.open, c.close, c.volume 
                    FROM s_{} AS c INNER JOIN name AS n 
                        ON c.code=n.code ORDER BY c.stime DESC LIMIT 365*20) AS t 
                            ORDER BY t.stime """.format(ucode)
    cursor.execute(sql)
    columns = ['code', 'lot', 'nmll', 'sdate', 'high', 'low', 'open', 'last', 'vol']
    data1[ucode] = pd.DataFrame(cursor.fetchall(), columns=columns)
conn.close()

In [41]:
data1

{'hsi':       code   lot  nmll       sdate      high       low      open      last  \
 0     .HSI  None  恒生指數  2000-01-03  17426.16  17057.70  17057.70  17369.63   
 1     .HSI  None  恒生指數  2000-01-04  17303.00  16933.52  17303.00  17072.82   
 2     .HSI  None  恒生指數  2000-01-05  16608.55  15688.49  16608.55  15846.72   
 3     .HSI  None  恒生指數  2000-01-06  15971.03  14763.97  15942.07  15153.23   
 4     .HSI  None  恒生指數  2000-01-07  15513.89  15108.75  15329.34  15405.63   
 ...    ...   ...   ...         ...       ...       ...       ...       ...   
 5229  .HSI  None  恒生指數  2021-03-17  29180.01  28780.23  28992.24  29034.12   
 5230  .HSI  None  恒生指數  2021-03-18  29596.58  29317.61  29317.61  29405.72   
 5231  .HSI  None  恒生指數  2021-03-19  29271.40  28737.54  29158.48  28990.94   
 5232  .HSI  None  恒生指數  2021-03-22  29139.07  28801.09  28801.09  28885.34   
 5233  .HSI  None  恒生指數  2021-03-23  29043.18  28376.22  29008.02  28497.38   
 
                vol  
 0              NaN  